In [6]:
"""
Author: Valentina Matos (Johns Hopkins - Wirtz/Kiemen Lab)
Date: May 29, 2024
"""
import os
import tensorflow as tf
import deeplabv3plus
from deeplabv3plus.train import Trainer

from tensorflow import keras
from deeplabv3plus.model.deeplabv3_plus import DeeplabV3Plus
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.optimizers import Adam
import pickle

In [7]:
import subprocess
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

try:
    output = subprocess.check_output(['nvidia-smi', '--query-gpu=gpu_name', '--format=csv,noheader'])
    gpu_name = output.decode('utf-8').strip()
    print(f"GPU Name: {gpu_name}")
    tf.config.set_visible_devices([], 'GPU')  # Use the first available GPU
except (subprocess.CalledProcessError, FileNotFoundError):
    print("Unable to retrieve GPU information")

Num GPUs Available:  1
GPU Name: NVIDIA GeForce RTX 3090


In [8]:
# Inputs
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\model test tiles'

In [9]:
# Load variables from pickle file
with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    sxy, classNames, nm = data['sxy'], data['classNames'], data['nm']

if 'net' in data:
    raise ValueError(f"A network has already been trained for model {nm}. Choose a new model name to retrain.")
else:
    # rest of the code here
    pass  # --delete pass when the rest of the code is placed below the else statement


In [10]:
# Paths to training and validation datasets:
classes = list(range(1, len(classNames)))
nmim = 'im'
nmlabel = 'label'

pthTrain = os.path.join(pthDL, 'training')
pthVal = os.path.join(pthDL, 'validation')

Train_HE = os.path.join(pthTrain, nmim)
Train_label = os.path.join(pthTrain, nmlabel)

Validation_HE = os.path.join(pthVal, nmim)
Validation_label = os.path.join(pthVal, nmlabel)

In [101]:
#ResNet50 model pre-trained on ImageNet
input_size = (1000, 1000, 3)  # Image dimensions
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_size)

model = DeepLabV3Plus(backbone='resnet50', num_classes=20)
input_shape = (1, 512, 512, 3)
input_tensor = tf.random.normal(input_shape)
result = model(input_tensor)  # build model by one forward pass
model.summary()

In [102]:
# Print layer names and output shapes
for layer in base_model.layers:
    print(f"{layer.name}: {layer.output_shape}")

input_19: [(None, 1000, 1000, 3)]
conv1_pad: (None, 1006, 1006, 3)
conv1_conv: (None, 500, 500, 64)
conv1_bn: (None, 500, 500, 64)
conv1_relu: (None, 500, 500, 64)
pool1_pad: (None, 502, 502, 64)
pool1_pool: (None, 250, 250, 64)
conv2_block1_1_conv: (None, 250, 250, 64)
conv2_block1_1_bn: (None, 250, 250, 64)
conv2_block1_1_relu: (None, 250, 250, 64)
conv2_block1_2_conv: (None, 250, 250, 64)
conv2_block1_2_bn: (None, 250, 250, 64)
conv2_block1_2_relu: (None, 250, 250, 64)
conv2_block1_0_conv: (None, 250, 250, 256)
conv2_block1_3_conv: (None, 250, 250, 256)
conv2_block1_0_bn: (None, 250, 250, 256)
conv2_block1_3_bn: (None, 250, 250, 256)
conv2_block1_add: (None, 250, 250, 256)
conv2_block1_out: (None, 250, 250, 256)
conv2_block2_1_conv: (None, 250, 250, 64)
conv2_block2_1_bn: (None, 250, 250, 64)
conv2_block2_1_relu: (None, 250, 250, 64)
conv2_block2_2_conv: (None, 250, 250, 64)
conv2_block2_2_bn: (None, 250, 250, 64)
conv2_block2_2_relu: (None, 250, 250, 64)
conv2_block2_3_conv: (None,

In [103]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Concatenate, Cropping2D
from tensorflow.keras.models import Model

# Load the ResNet50 model with pre-trained weights
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(1000, 1000, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Decoder path with transposed convolutions
x = base_model.output  # Start with the output of the ResNet50
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same')(x)

# Ensure matching dimensions before concatenation with 'conv4_block6_out'
conv4_block6_out = base_model.get_layer('conv4_block6_out').output
x = Cropping2D(((1, 0), (1, 0)))(x)  # Adjust dimensions to match
x = Concatenate()([x, conv4_block6_out])

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same')(x)

# Ensure matching dimensions before concatenation with 'conv3_block4_out'
conv3_block4_out = base_model.get_layer('conv3_block4_out').output
x = Cropping2D(((1, 0), (1, 0)))(x)  # Adjust dimensions to match
x = Concatenate()([x, conv3_block4_out])

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(x)

# Ensure matching dimensions before concatenation with 'conv2_block3_out'
conv2_block3_out = base_model.get_layer('conv2_block3_out').output
x = Concatenate()([x, conv2_block3_out])

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same')(x)

# Ensure matching dimensions before concatenation with 'conv1_conv'
conv1_conv = base_model.get_layer('conv1_conv').output
x = Concatenate()([x, conv1_conv])

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

# Final convolutional layer for class prediction
output = Conv2D(len(classNames), (1, 1), activation='softmax')(x)


In [104]:
# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Print model summary
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 1000, 1000,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 1006, 1006,   0           ['input_20[0][0]']               
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, 500, 500, 64  9472        ['conv1_pad[0][0]']              
                                )                                                           

In [105]:
# Fine-tune the entire model
#  sets all layers of the pre-trained ResNet50 model as trainable. By default, when using a pre-trained model, the base layers are frozen (not trainable) to preserve the learned weights. Setting layer.trainable = True for all layers allows the entire model to be fine-tuned on your specific task and dataset.
for layer in base_model.layers:
    layer.trainable = True

In [106]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [107]:
### Prepare the data generators

# ImageDataGenerator for training data with rescaling
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalizes the pixel values to [0, 1]
# ImageDataGenerator for validation data with rescaling
val_datagen = ImageDataGenerator(rescale=1./255)    # Normalizes the pixel values to [0, 1]


classes = [str(cls) for cls in classes] #convert classes for string format

train_generator = train_datagen.flow_from_directory(
    Train_HE,                # Path to the training data directory containing subdirectories of images
    target_size=(1000, 1000),# Resize all images to 1000x1000 pixels
    batch_size=4,            # Number of images to be yielded from the generator per batch
    class_mode=None,         # Don't return the labels (since it's an autoencoder task or a task where labels are not needed here)
    color_mode='rgb',        # Read images in RGB mode
    classes=classes,         # List of subdirectory names to consider as valid labels
    seed=42                  # Random seed for reproducibility
)


val_generator = val_datagen.flow_from_directory(
    Validation_HE,
    target_size=(1000, 1000),
    batch_size=4,
    class_mode=None,
    color_mode='rgb',
    classes=classes,
    seed=42
)

# Generate batches of label data with real-time data augmentation.
train_label_generator = train_datagen.flow_from_directory(
    Train_label,             # Path to the directory containing training labels
    target_size=(1000, 1000),# Resize all labels to 1000x1000 pixels
    batch_size=4,            # Number of labels to be yielded from the generator per batch
    class_mode='categorical',# Return the labels as one-hot encoded arrays
    color_mode='grayscale',  # Read labels in grayscale mode
    classes=classes,         # List of subdirectory names to consider as classes
    seed=42                  # Random seed for reproducibility
)

val_label_generator = val_datagen.flow_from_directory(
    Validation_label,
    target_size=(1000, 1000),
    batch_size=4,
    class_mode='categorical',
    color_mode='grayscale',
    classes=classes,
    seed=42
)

Found 0 images belonging to 11 classes.
Found 0 images belonging to 11 classes.
Found 0 images belonging to 11 classes.
Found 0 images belonging to 11 classes.


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np

def custom_image_generator(directory, target_size, batch_size, color_mode, classes, seed):
    np.random.seed(seed)
    while True:
        batch_features = []
        batch_labels = []
        for _ in range(batch_size):
            # Randomly select a class
            class_name = np.random.choice(classes)
            class_dir = os.path.join(directory, class_name)
            # Randomly select an image from the class directory
            image_name = np.random.choice(os.listdir(class_dir))
            image_path = os.path.join(class_dir, image_name)
            
            # Load the image
            img = load_img(image_path, target_size=target_size, color_mode=color_mode)
            img_array = img_to_array(img) / 255.0
            
            batch_features.append(img_array)
            # One-hot encode the label
            label = np.zeros(len(classes))
            label[classes.index(class_name)] = 1
            batch_labels.append(label)
        
        yield np.array(batch_features), np.array(batch_labels)

# Convert classes to string format
class_dirs = [str(cls) for cls in classes]

# Create custom generators
train_generator = custom_image_generator(
    Train_HE, 
    target_size=(1000, 1000), 
    batch_size=4, 
    color_mode='rgb', 
    classes=class_dirs, 
    seed=42
)

val_generator = custom_image_generator(
    Validation_HE, 
    target_size=(1000, 1000), 
    batch_size=4, 
    color_mode='rgb', 
    classes=class_dirs, 
    seed=42
)

train_label_generator = custom_image_generator(
    Train_label, 
    target_size=(1000, 1000), 
    batch_size=4, 
    color_mode='grayscale', 
    classes=class_dirs, 
    seed=42
)

val_label_generator = custom_image_generator(
    Validation_label, 
    target_size=(1000, 1000), 
    batch_size=4, 
    color_mode='grayscale', 
    classes=class_dirs, 
    seed=42
)


In [108]:
print(Train_HE)

\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\model test tiles\training\im


In [78]:
# Define the learning rate schedule
def step_decay(epoch):
    initial_lr = 0.001
    drop_rate = 0.5
    epoch_drop = 5
    lrate = initial_lr * drop_rate ** (epoch // epoch_drop)
    return lrate

# Create the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(step_decay)

In [79]:
# Create the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

In [80]:
# Train the model
history = model.fit(
    train_generator,
    train_label_generator,
    steps_per_epoch=len(train_generator),
    epochs=8,
    validation_data=(val_generator, val_label_generator),
    validation_steps=len(val_generator),
    callbacks=[lr_scheduler, early_stopping]
)

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.

In [ ]:
# Save the model
print('Saving model weights...')
data['net'] = model.get_weights()  # Get the model weights
data['history'] = history.history  # Get the model history

with open(os.path.join(pthDL, 'net.pkl'), 'wb') as f:
    pickle.dump(data, f)

In [17]:
# Load the model history from the pickle file
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    model_history = data['history']
    
history_df = pd.DataFrame(model_history)
# Use seaborn color palette
sns.set_palette("colorblind")

# Plot loss and accuracy in a single figure
plt.figure(figsize=(12, 6))

# Plot loss
plt.subplot(1, 2, 1)
sns.lineplot(data=history_df, x=history_df.index, y='loss', label='Training Loss')
sns.lineplot(data=history_df, x=history_df.index, y='val_loss', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)

# Plot accuracy
plt.subplot(1, 2, 2)
sns.lineplot(data=history_df, x=history_df.index, y='accuracy', label='Training Accuracy')
sns.lineplot(data=history_df, x=history_df.index, y='val_accuracy', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '\\\\10.99.68.52\\Kiemendata\\Valentina Matos\\coda to python\\test model\\04_19_2024\\net.pkl'

In [90]:
%reset